In [1]:
import numpy as np
import cv2

In [2]:
# 1. Generating Artificial Data

# Intrinsic matrices for two cameras
K1 = np.array([
    [800, 0, 640],
    [0, 800, 360],
    [0, 0, 1]
])

K2 = np.array([
    [805, 0, 642],
    [0, 805, 362],
    [0, 0, 1]
])

# Generate random 3D points as ground truth
num_points = 10
X_3D = np.hstack((np.random.randint(0, 50, (num_points, 1)),
                  np.random.randint(0, 50, (num_points, 1)),
                  np.random.randint(50, 100, (num_points, 1))))

# Camera 1 is at the origin of the global coordinate system
P1 = K1 @ np.hstack((np.eye(3), np.zeros((3, 1))))

# Define a relative rotation and translation for Camera 2
R = cv2.Rodrigues(np.array([0.1, 0.2, 0.3]))[0]
t = np.array([[10], [5], [2]])

P2 = K2 @ np.hstack((R, t))

# Convert 3D points to homogeneous coordinates for projection
X_3D_homogeneous = np.hstack([X_3D, np.ones((num_points, 1))])

# Project the 3D points to the images
points1_homogeneous = P1 @ X_3D_homogeneous.T
points1 = (points1_homogeneous[:2, :] / points1_homogeneous[2, :]).T

points2_homogeneous = P2 @ X_3D_homogeneous.T
points2 = (points2_homogeneous[:2, :] / points2_homogeneous[2, :]).T


In [3]:
# 2. Stereo Triangulation

# Compute the fundamental matrix
F, _ = cv2.findFundamentalMat(points1, points2, cv2.FM_LMEDS)

# Compute the essential matrix
E = K1.T @ F @ K2

# Recover pose from essential matrix
_, R_recovered, t_recovered, _ = cv2.recoverPose(E, points1, points2, K1)

# Projection matrix for the second camera
P2_recovered = K2 @ np.hstack((R_recovered, t_recovered))

# Triangulate points
points_4D_hom = cv2.triangulatePoints(P1, P2_recovered, points1.T, points2.T)
points_3D_recovered = (points_4D_hom[:3, :] / points_4D_hom[3, :]).T


In [4]:
# 3. Comparison

# Reprojection errors
reprojected_points1_homogeneous = P1 @ points_4D_hom
reprojected_points1 = (reprojected_points1_homogeneous[:2, :] / reprojected_points1_homogeneous[2, :]).T

reprojected_points2_homogeneous = P2_recovered @ points_4D_hom
reprojected_points2 = (reprojected_points2_homogeneous[:2, :] / reprojected_points2_homogeneous[2, :]).T

error1 = np.sqrt(np.mean(np.sum((points1 - reprojected_points1)**2, axis=1)))
error2 = np.sqrt(np.mean(np.sum((points2 - reprojected_points2)**2, axis=1)))

total_reprojection_error = (error1 + error2) / 2

print(f"Mean Reprojection Error: {total_reprojection_error:.2f}")

Mean Reprojection Error: 3.80


In [5]:
points1

array([[ 876.73469388,  572.24489796],
       [ 812.04301075,  721.29032258],
       [1040.        ,  465.88235294],
       [ 699.25925926,  493.33333333],
       [ 847.40740741,  616.79012346],
       [1145.26315789,  935.43859649],
       [ 933.33333333,  906.66666667],
       [ 986.66666667,  360.        ],
       [1084.44444444,  671.11111111],
       [ 831.04477612,  467.46268657]])

In [6]:
reprojected_points1

array([[ 877.18085726,  568.82210247],
       [ 812.45575386,  716.94834658],
       [1040.40104815,  463.37895821],
       [ 699.70138893,  490.17698166],
       [ 847.85153358,  613.06465043],
       [1145.4980804 ,  930.9623794 ],
       [ 933.66325303,  900.82611095],
       [ 986.9963346 ,  358.18737544],
       [1084.78613447,  668.10449124],
       [ 831.47040942,  464.65071529]])

In [7]:
points2

array([[1065.9320302 ,  629.9005939 ],
       [ 952.01778694,  742.64875699],
       [1322.8424143 ,  606.43121351],
       [ 971.43890775,  524.1012012 ],
       [1035.0536362 ,  667.59361723],
       [1271.82034248, 1091.07527291],
       [1053.69110347,  968.77046742],
       [1318.9136833 ,  488.81907738],
       [1283.45979963,  817.95532864],
       [1089.97051511,  531.8803595 ]])

In [8]:
reprojected_points2

array([[1064.37093237,  633.15489505],
       [ 950.08634183,  746.74820223],
       [1321.65473659,  608.8687801 ],
       [ 970.04303697,  526.99774929],
       [1033.3723384 ,  671.11986858],
       [1269.74613793, 1095.57251201],
       [1051.08133351,  974.42463153],
       [1318.05432545,  490.56132759],
       [1282.04795641,  820.92370287],
       [1088.69011585,  534.52069536]])